# Y~Weather Hilbert Curve 2d

> 

In [ ]:
import numpy as np
import pandas as pd

from EnvDL.core import ensure_dir_path_exists 
from EnvDL.dlfn import g2fc_datawrapper, BigDataset, plDNN_general
from EnvDL.dlfn import ResNet2d, BasicBlock2d

import torch
import torch.nn.functional as F # F.mse_loss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger

In [ ]:
# Run settings: 
max_epoch = 10
batch_size = 50

In [ ]:
use_gpu_num = 0

device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

In [ ]:
cache_path = '../nbs_artifacts/03.12_g2fc_W_Hilbert_conv2d_resnet.ipynb/'
ensure_dir_path_exists(dir_path = cache_path)

## Load data

In [ ]:
X = g2fc_datawrapper()
X.set_split()
X.load_all(name_list = ['obs_env_lookup', 'YMat', 'PlantHarvest', 'WMat_hilb',], store=True) 

X.calc_cs('YMat', version = 'np', filter = 'val:train')
X.calc_cs('WMat_hilb',            filter = 'val:train', filter_lookup= 'obs_env_lookup')

In [ ]:
training_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = False,
    lookup_obs = X.get('val:train',      ops_string='   asarray from_numpy             '),
    lookup_env = X.get('obs_env_lookup', ops_string='   asarray from_numpy             '),
    y =          X.get('YMat',           ops_string='cs asarray from_numpy float cuda:0')[:, None],
    W =          X.get('WMat_hilb',      ops_string='cs         from_numpy float cuda:0'),
    P =          X.get('PlantHarvest',   ops_string='           from_numpy             '),
    W_type = 'hilbert'
    ),
    batch_size = batch_size,
    shuffle = True
)


validation_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = False,
    lookup_obs = X.get('val:test',       ops_string='   asarray from_numpy             '),
    lookup_env = X.get('obs_env_lookup', ops_string='   asarray from_numpy             '),
    y =          X.get('YMat',           ops_string='cs asarray from_numpy float cuda:0')[:, None],
    W =          X.get('WMat_hilb',      ops_string='cs         from_numpy float cuda:0'),
    P =          X.get('PlantHarvest',   ops_string='           from_numpy             '),
    W_type = 'hilbert'
    ),
    batch_size = batch_size,
    shuffle = False
)

## Test Models

In [ ]:
model = ResNet2d(
        block = BasicBlock2d, #: Type[Union[BasicBlock, Bottleneck]],
        layers = [2, 2, 2, 2], #: List[int],
        # num_classes: int = 1000,
        zero_init_residual = False,
        groups = 1,
        width_per_group = 64,
        replace_stride_with_dilation = None,
        norm_layer = None,
        input_channels = 17
    ).to('cuda')

DNNW = plDNN_general(model)     
optimizer = DNNW.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="w-hilb-res-4rep2-from-pytorch")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNW, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)


In [ ]:
model = ResNet2d(
        block = BasicBlock2d, #: Type[Union[BasicBlock, Bottleneck]],
        layers = [4, 4, 4, 4], #: List[int],
        # num_classes: int = 1000,
        zero_init_residual = False,
        groups = 1,
        width_per_group = 64,
        replace_stride_with_dilation = None,
        norm_layer = None,
        input_channels = 17
    ).to('cuda')

DNNW = plDNN_general(model)     
optimizer = DNNW.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="w-hilb-res-4rep4-from-pytorch")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNW, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)